# Proper nouns with or without definite articles (Nestle1904GBI)

## Table of content <a class="anchor" id="TOC">
* [1 - Introduction](#first-bullet)
    
    * [1.1 - Why is this relevant?](#bullet1_1)
    
    * [1.2 - Translating into Text-Fabric queries](#bullet1_2)
    
    * [1.3 - Notes](#bullet1_3)
    
* [2 - Load Text-Fabric app and data](#second-bullet)
* [3 - Performing the queries](#third-bullet)
    * [3.1 - TBD](#bullet3_1)
    * [3.2 - TBD](#bullet3_2)

# 1 - Introduction <a class="anchor" id="first-bullet"></a>

In this Jupyter NoteBook we will examine roper nouns with or without definite articles in the Greek New Testament.

## 1.1 - Why is this relevant? <a class="anchor" id="#bullet1_1"></a>

In the Greek New Testament, the presence or absence of definite articles with proper nouns can carry specific grammatical and semantic implications. The pressence or absence of the article can make a difference for a few reasons, e.g.:

* Identifiability: The presence of a definite article (e.g., ὁ, ἡ, τὸ) before a proper noun indicates that the noun refers to a specific, identifiable entity. It often implies that the noun is known to both the speaker and the audience. On the other hand, the absence of the definite article suggests a more general or less specific reference.

* Emphasis: The use or omission of the definite article can be employed for emphasis or to highlight certain aspects of the proper noun. When the definite article is present, it draws attention to the individual or highlight particular qualities or characteristics. Without the definite article, the focus may shift more to the general category or class to which the noun belongs.

* Unique versus Common Names: Proper nouns in Greek can be classified as either unique or common names. Unique names refer to specific individuals or places, such as Ἰησοῦς or Ἱεροσόλυμα. These often appear with the definite article when referring to a well-known individual or location. Common names, on the other hand, are more general and do not require the definite article. For example, ἄνθρωπος can refer to any man in general.

* Anaphoric References: In Greek, the presence or absence of the definite article can indicate anaphoric references, referring back to a previous mention. The use of the definite article can signal that the proper noun is referring to someone or something previously introduced or discussed, providing continuity and clarity in the discourse.


## 1.2 - Translating into Text-Fabric queries <a class="anchor" id="#bullet1_2"></a>

The following examples gather pressence and absence of definive article with proper nouns.

## 1.3 - Notes <a class="anchor" id="#bullet1_3"></a>

Thanks to Prof. Willem van Peursen for pointing me to this question.

# 2 - Load Text-Fabric app and data <a class="anchor" id="second-bullet"></a>
##### [Back to TOC](#TOC)

In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
# Loading the New Testament TextFabric code
# Note: it is assumed Text-Fabric is installed in your environment.

from tf.fabric import Fabric
from tf.app import use

In [3]:
# load the app and data
N1904 = use ("tonyjurg/Nestle1904GBI:latest", hoist=globals())

**Locating corpus resources ...**

The requested data is not available offline
	~/text-fabric-data/github/tonyjurg/Nestle1904GBI/tf/0.3 not found


Name,# of nodes,# slots/node,% coverage
book,27,5102.93,100
chapter,260,529.92,100
sentence,5720,24.09,100
verse,7944,17.34,100
clause,16124,8.54,100
phrase,73547,1.87,100
word,137779,1.00,100


# 3 - Performing the queries <a class="anchor" id="third-bullet"></a>

## 3.1 - Determine the conditions <a class="anchor" id="bullet3_1"></a>
##### [Back to TOC](#TOC)

This code will produce .